# ICU Mortality Model - Cohort Generation

This notebook generates the ICU cohort for mortality prediction modeling following the PRD requirements.

## Objective
Generate a cohort table containing:
- `hospitalization_id`
- `start_dttm`: ICU admission timestamp
- `hour_24_start_dttm`: first ICU hour (may equal start_dttm)
- `hour_24_end_dttm`: end of the first 24 hours
- `disposition`: binary outcome (1 = expired, 0 = survived)

## Cohort Criteria
- First 24 hours of first ICU stay
- Exclude re-admissions and ICU readmissions
- ICU-OR-ICU sequences treated as continuous ICU stay
- Minimum 24-hour ICU stay
- Adults (≥18 years)
- 2020-2021 data

## Setup and Configuration

In [18]:
import sys
import os
sys.path.append(os.path.join('..', 'src'))

import pandas as pd
import numpy as np
from pyclif import CLIF
import json
import warnings
warnings.filterwarnings('ignore')

print("=== ICU Mortality Model - Cohort Generation ===")
print("Setting up environment...")

=== ICU Mortality Model - Cohort Generation ===
Setting up environment...


In [19]:
def load_config():
    """Load configuration from config.json"""
    config_path = os.path.join( "config_demo.json")
    
    if os.path.exists(config_path):
        with open(config_path, 'r') as file:
            config = json.load(file)
        print("✅ Loaded configuration from config.json")
    else:
        raise FileNotFoundError("Configuration file not found. Please create config.json based on the config_template.")
    
    return config

# Load configuration
config = load_config()
print(f"Site: {config['site']}")
print(f"Data path: {config['clif2_path']}")
print(f"File type: {config['filetype']}")

✅ Loaded configuration from config.json
Site: MIMIC
Data path: /Users/sudo_sage/Documents/WORK/clif_mimic
File type: parquet


In [20]:
# Initialize pyCLIF
clif = CLIF(
    data_dir=config['clif2_path'],
    filetype=config['filetype'],
    timezone="US/Eastern"
)

print("✅ pyCLIF initialized successfully")

CLIF Object Initialized.
✅ pyCLIF initialized successfully


## Data Loading and Preparation

In [21]:
# Load required tables using pyCLIF
print("Loading required tables...")
clif.initialize(["adt", "hospitalization", "patient"])

# Load ADT data
adt_df = clif.adt.df.copy()
print(f"ADT data loaded: {len(adt_df)} records")

# Load hospitalization data
hosp_df = clif.hospitalization.df.copy()
print(f"Hospitalization data loaded: {len(hosp_df)} records")

# Load patient data
patient_df = clif.patient.df.copy()
print(f"Patient data loaded: {len(patient_df)} records")

Loading required tables...
Loading clif_adt.parquet
Data loaded successfully from clif_adt.parquet
Validation completed with 3 error(s). See `errors` attribute.
Loading clif_hospitalization.parquet
Data loaded successfully from clif_hospitalization.parquet
Validation completed with 1 error(s). See `errors` attribute.
Loading clif_patient.parquet
Data loaded successfully from clif_patient.parquet
Validation completed with 8 error(s). See `errors` attribute.
ADT data loaded: 1458408 records
Hospitalization data loaded: 546028 records
Patient data loaded: 364627 records


In [22]:
adt_df.location_category.value_counts()

location_category
ward          640846
ed            476770
icu           118077
other          71399
stepdown       70482
procedural     34958
l&d            29003
psych          16873
Name: count, dtype: int64

In [23]:
# Prepare data for cohort generation
print("Preparing data for cohort generation...")

# Merge ADT with hospitalization data
icu_data = pd.merge(
    adt_df[['hospitalization_id', 'location_category', 'in_dttm', 'out_dttm']],
    hosp_df[['patient_id', 'hospitalization_id', 'age_at_admission', 'discharge_category', 'admission_dttm']],
    on='hospitalization_id',
    how='left'
)

print(f"Merged data: {len(icu_data)} records")

# Convert datetime columns
datetime_cols = ['in_dttm', 'out_dttm', 'admission_dttm']
for col in datetime_cols:
    icu_data[col] = pd.to_datetime(icu_data[col])

# Handle location categories (convert procedural to OR as in Inference_py.ipynb)
icu_data.loc[icu_data['location_category'] == 'procedural', 'location_category'] = 'OR'
icu_data['location_category'] = icu_data['location_category'].str.upper()

print("✅ Data preparation completed")

Preparing data for cohort generation...
Merged data: 1458408 records
✅ Data preparation completed


## ICU Cohort Selection

In [24]:
icu_data.head()

,hospitalization_id,location_category,in_dttm,out_dttm,patient_id,age_at_admission,discharge_category,admission_dttm
0,22595853,ED,2180-05-07 00:17:00+00:00,2180-05-07 04:30:00+00:00,10000032,52,Home,2180-05-07 03:23:00+00:00
1,22595853,WARD,2180-05-07 04:30:00+00:00,2180-05-07 22:21:27+00:00,10000032,52,Home,2180-05-07 03:23:00+00:00
2,22841357,WARD,2180-06-27 02:31:00+00:00,2180-06-27 23:49:12+00:00,10000032,52,Home,2180-06-26 23:27:00+00:00
3,22841357,ED,2180-06-26 20:54:00+00:00,2180-06-27 02:31:00+00:00,10000032,52,Home,2180-06-26 23:27:00+00:00
4,25742920,WARD,2180-08-06 06:44:00+00:00,2180-08-07 22:50:44+00:00,10000032,52,Hospice,2180-08-06 04:44:00+00:00


In [25]:
# Apply initial filters
print("Applying initial cohort filters...")

# Filter for ICU admissions within 48 hours of hospital admission
icu_48hr_check = icu_data[
    (icu_data['location_category'] == 'ICU') &
    (icu_data['in_dttm'] >= icu_data['admission_dttm']) &
    (icu_data['in_dttm'] <= icu_data['admission_dttm'] + pd.Timedelta(hours=48)) &
   # (icu_data['admission_dttm'].dt.year >= 2020) & (icu_data['admission_dttm'].dt.year <= 2021) &
    (icu_data['age_at_admission'] >= 18) & (icu_data['age_at_admission'].notna())
]['hospitalization_id'].unique()

print(f"Hospitalizations with ICU within 48hr: {len(icu_48hr_check)}")

# Filter to relevant encounters and extend to 72 hours for location tracking
icu_data = icu_data[
    icu_data['hospitalization_id'].isin(icu_48hr_check) &
    (icu_data['in_dttm'] <= icu_data['admission_dttm'] + pd.Timedelta(hours=72))
].reset_index(drop=True)

print(f"Filtered data for processing: {len(icu_data)} records")

Applying initial cohort filters...
Hospitalizations with ICU within 48hr: 73430
Filtered data for processing: 237855 records


In [26]:
# Process ICU-OR-ICU sequences (treat as continuous ICU)
print("Processing ICU-OR-ICU sequences...")

# Sort by admission time and create ranking
icu_data = icu_data.sort_values(by=['in_dttm']).reset_index(drop=True)
icu_data["RANK"] = icu_data.sort_values(by=['in_dttm'], ascending=True).groupby("hospitalization_id")["in_dttm"].rank(method="first", ascending=True).astype(int)

# Find minimum ICU rank for each hospitalization
min_icu = icu_data[icu_data['location_category'] == 'ICU'].groupby('hospitalization_id')['RANK'].min()
icu_data = pd.merge(icu_data, pd.DataFrame(zip(min_icu.index, min_icu.values), columns=['hospitalization_id', 'min_icu']), on='hospitalization_id', how='left')

# Filter to locations from first ICU onward
icu_data = icu_data[icu_data['RANK'] >= icu_data['min_icu']].reset_index(drop=True)

# Convert OR to ICU for continuity (ICU-OR-ICU treated as continuous ICU)
icu_data.loc[icu_data['location_category'] == 'OR', 'location_category'] = 'ICU'

print(f"After ICU-OR-ICU processing: {len(icu_data)} records")

Processing ICU-OR-ICU sequences...
After ICU-OR-ICU processing: 148501 records


In [27]:
# Group consecutive ICU locations
print("Grouping consecutive ICU locations...")

# Create groups for consecutive locations
icu_data['group_id'] = (icu_data.groupby('hospitalization_id')['location_category'].shift() != icu_data['location_category']).astype(int)
icu_data['group_id'] = icu_data.sort_values(by=['in_dttm'], ascending=True).groupby('hospitalization_id')['group_id'].cumsum()

# Aggregate by groups
icu_data = icu_data.sort_values(by=['in_dttm'], ascending=True).groupby(['patient_id', 'hospitalization_id', 'location_category', 'group_id']).agg(
    min_in_dttm=('in_dttm', 'min'),
    max_out_dttm=('out_dttm', 'max'),
    admission_dttm=('admission_dttm', 'first'),
    age=('age_at_admission', 'first'),
    dispo=('discharge_category', 'first')
).reset_index()

print(f"Grouped data: {len(icu_data)} records")

Grouping consecutive ICU locations...
Grouped data: 125005 records


In [28]:
# Apply final cohort criteria
print("Applying final cohort criteria...")

# Find minimum ICU group for each hospitalization
min_icu_group = icu_data[icu_data['location_category'] == 'ICU'].groupby('hospitalization_id')['group_id'].min()
icu_data = pd.merge(icu_data, pd.DataFrame(zip(min_icu_group.index, min_icu_group.values), columns=['hospitalization_id', 'min_icu_group']), on='hospitalization_id', how='left')

# Filter to first ICU stay with minimum 24-hour duration
icu_data = icu_data[
    (icu_data['min_icu_group'] == icu_data['group_id']) &
    (icu_data['max_out_dttm'] - icu_data['min_in_dttm'] >= pd.Timedelta(hours=24))
].reset_index(drop=True)

print(f"Final cohort before demographics: {len(icu_data)} records")

# Add 24-hour endpoint
icu_data['after_24hr'] = icu_data['min_in_dttm'] + pd.Timedelta(hours=24)

# Select required columns
icu_data = icu_data[['patient_id', 'hospitalization_id', 'min_in_dttm', 'max_out_dttm', 'after_24hr', 'age', 'dispo']]

print("✅ ICU cohort criteria applied")

Applying final cohort criteria...
Final cohort before demographics: 54509 records
✅ ICU cohort criteria applied


## Add Demographics and Create Final Cohort

In [29]:
# Add patient demographics
print("Adding patient demographics...")

# Rename columns for consistency with CLIF 2.0
patient_df_clean = patient_df.rename(columns={
    'race_category': 'race',
    'ethnicity_category': 'ethnicity',
    'sex_category': 'sex'
})

# Merge with patient data
icu_data = pd.merge(
    icu_data,
    patient_df_clean[['patient_id', 'sex', 'ethnicity', 'race']],
    on='patient_id',
    how='left'
)

# Filter out records with missing sex (data quality)
icu_data = icu_data[~icu_data['sex'].isna()].reset_index(drop=True)

print(f"Final cohort with demographics: {len(icu_data)} records")

Adding patient demographics...
Final cohort with demographics: 54509 records


In [30]:
icu_data.head()

,patient_id,hospitalization_id,min_in_dttm,max_out_dttm,after_24hr,age,dispo,sex,ethnicity,race
0,10000690,25860671,2150-11-03 00:37:00+00:00,2150-11-06 22:03:17+00:00,2150-11-04 00:37:00+00:00,86,Acute Inpatient Rehab Facility,Female,Non-Hispanic,White
1,10001217,24597018,2157-11-21 00:18:02+00:00,2157-11-22 03:08:00+00:00,2157-11-22 00:18:02+00:00,55,Home,Female,Non-Hispanic,White
2,10001725,25563031,2110-04-11 20:52:22+00:00,2110-04-13 04:59:56+00:00,2110-04-12 20:52:22+00:00,46,Home,Female,Non-Hispanic,White
3,10002013,23581541,2160-05-18 15:00:53+00:00,2160-05-19 22:33:33+00:00,2160-05-19 15:00:53+00:00,57,Home,Female,Non-Hispanic,White
4,10002114,27793700,2162-02-18 04:30:00+00:00,2162-02-21 02:16:27+00:00,2162-02-19 04:30:00+00:00,56,Home,Male,Unknown,Unknown


In [31]:
# Create final cohort table with required columns
print("Creating final cohort table...")

# Create disposition binary variable (1 = expired, 0 = survived)
icu_data['disposition'] = (icu_data['dispo'].fillna('Other').str.contains('dead|expired|death|died', case=False, regex=True)).astype(int)

# Create final cohort with PRD required columns
cohort_final = icu_data[[
    'hospitalization_id',
    'min_in_dttm',     # start_dttm
    'after_24hr',      # hour_24_end_dttm
    'disposition'
]].rename(columns={
    'min_in_dttm': 'start_dttm',
    'after_24hr': 'hour_24_end_dttm'
})

# Add hour_24_start_dttm (same as start_dttm for our cohort)
cohort_final['hour_24_start_dttm'] = cohort_final['start_dttm']

# Reorder columns as per PRD
cohort_final = cohort_final[[
    'hospitalization_id',
    'start_dttm',
    'hour_24_start_dttm',
    'hour_24_end_dttm',
    'disposition'
]]

print(f"✅ Final cohort created: {len(cohort_final)} hospitalizations")
print(f"Mortality rate: {cohort_final['disposition'].mean():.3f}")

Creating final cohort table...
✅ Final cohort created: 54509 hospitalizations
Mortality rate: 0.110


In [32]:
cohort_final['disposition'].value_counts()*100/cohort_final.shape[0]

disposition
0    88.96696
1    11.03304
Name: count, dtype: float64

## Cohort Summary and Validation

In [33]:
# Display cohort summary
print("=== ICU Cohort Summary ===")
print(f"Total hospitalizations: {len(cohort_final):,}")
print(f"Mortality rate: {cohort_final['disposition'].mean():.3f} ({cohort_final['disposition'].sum():,} deaths)")
print(f"Survival rate: {1 - cohort_final['disposition'].mean():.3f} ({(cohort_final['disposition'] == 0).sum():,} survivors)")

# Time range analysis
print(f"\n=== Time Range Analysis ===")
print(f"Cohort start date: {cohort_final['start_dttm'].min()}")
print(f"Cohort end date: {cohort_final['start_dttm'].max()}")
print(f"24-hour window duration: {(cohort_final['hour_24_end_dttm'] - cohort_final['hour_24_start_dttm']).iloc[0]}")

# Validation checks
print(f"\n=== Validation Checks ===")
print(f"All 24-hour windows are exactly 24 hours: {((cohort_final['hour_24_end_dttm'] - cohort_final['hour_24_start_dttm']).dt.total_seconds() == 24*3600).all()}")
print(f"No missing hospitalization IDs: {cohort_final['hospitalization_id'].isna().sum() == 0}")
print(f"All start times before end times: {(cohort_final['start_dttm'] <= cohort_final['hour_24_end_dttm']).all()}")

=== ICU Cohort Summary ===
Total hospitalizations: 54,509
Mortality rate: 0.110 (6,014 deaths)
Survival rate: 0.890 (48,495 survivors)

=== Time Range Analysis ===
Cohort start date: 2105-10-04 22:27:12+00:00
Cohort end date: 2214-05-03 22:09:18+00:00
24-hour window duration: 1 days 00:00:00

=== Validation Checks ===
All 24-hour windows are exactly 24 hours: True
No missing hospitalization IDs: True
All start times before end times: True


## Save Cohort to Output Directory

In [34]:
# Save cohort to output/preprocessing directory
output_path = os.path.join('..', 'output', 'preprocessing', 'icu_cohort.parquet')
cohort_final.to_parquet(output_path, index=False)

print(f"✅ Cohort saved to: {output_path}")
print(f"File size: {os.path.getsize(output_path) / 1024:.1f} KB")
print(f"Shape: {cohort_final.shape}")

# Save additional metadata
metadata = {
    'cohort_size': len(cohort_final),
    'mortality_rate': float(cohort_final['disposition'].mean()),
    'date_range': {
        'start': cohort_final['start_dttm'].min().isoformat(),
        'end': cohort_final['start_dttm'].max().isoformat()
    },
    'criteria': {
        'min_age': 18,
        'years': '2020-2021',
        'icu_window': '48_hours_from_admission',
        'min_icu_duration': '24_hours',
        'icu_or_icu_handling': 'continuous_icu'
    }
}

metadata_path = os.path.join('..', 'output', 'preprocessing', 'cohort_metadata.json')
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✅ Metadata saved to: {metadata_path}")
print("\n🎉 Cohort generation completed successfully!")

✅ Cohort saved to: ../output/preprocessing/icu_cohort.parquet
File size: 2091.1 KB
Shape: (54509, 5)
✅ Metadata saved to: ../output/preprocessing/cohort_metadata.json

🎉 Cohort generation completed successfully!
